In [217]:
### TODO import libraries and apply overlay
from PIL import Image
import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pynq import allocate, Overlay

In [218]:
out_img_h = 64
out_img_w = 64
image_list = []

In [219]:
### TODO download cat images into the folder `cats/`
def fetch_images():
    for fName in glob.glob('images/*.jpg'):
        im = Image.open(fName)
        image_list.append(im)

In [220]:
# image_in, image_out: PIL.Image
# image_out_size: (width, height)
def fpga_resize(image_in, image_out_size):
    ### YOUR CODE HERE
    ## Initialisation of DMA
    resize_design = Overlay("resizer.bit")
    dma = resize_design.axi_dma_0
    resizer = resize_design.resize_accel_0

    img_ip_w, img_ip_h = image_in.size
    ## img_op_h, img_op_w = image_out_size.size
    in_buffer = allocate (shape=(img_ip_h,img_ip_w,3), dtype=np.uint8, cacheable=1)
    out_buffer = allocate (shape=(out_img_h,out_img_w,3), dtype=np.uint8, cacheable=1)
    
    in_buffer[:] = np.array(image_in)
 
    ##Run the resizer function
    def run_kernel():
        dma.sendchannel.transfer(in_buffer)
        dma.recvchannel.transfer(out_buffer)    
        resizer.write(0x00,0x81) # start
        dma.sendchannel.wait()
        dma.recvchannel.wait()
    
    ##Initialize resizer
    resizer.write(0x10, img_ip_h)
    resizer.write(0x18, img_ip_w)
    resizer.write(0x20, 64)
    resizer.write(0x28, 64)
    
    run_kernel()
    image_out = Image.fromarray(out_buffer)
    
    del in_buffer
    del out_buffer
    
    return image_out

In [221]:
def fpga_resize_images(image_paths):
    ### YOUR CODE HERE
    images = []
    for fName in glob.glob(image_paths):
        im = Image.open(fName)
        temp = fpga_resize(im, 0)
        images.append(temp)
    
    return images

In [222]:
def cpu_resize_images(image_paths):
    ### YOUR CODE HERE
    images = []
    for fName in glob.glob(image_paths):
        im = Image.open(fName)
        images.append(im.resize((64,64), Image.BILINEAR))
                           
    return images

In [223]:
# TODO runtime comparison

In [224]:
image_paths = 'images/c*.jpg'

In [225]:
%%timeit -n1 -r1
fpga_img_list = fpga_resize_images(image_paths)

1 loop, best of 1: 5.71 s per loop


In [226]:
%%timeit -n1 -r1
cpu_img_list = cpu_resize_images(image_paths)

1 loop, best of 1: 82.8 ms per loop


In [227]:
# TODO visualization of comparison values using plots